# Text Generator

Read in the Frost poetry text file to use in a **second-order Markov model**. This is an example of training the Markov model without matrices, but using dictionaries instead (dictionaries take up less space).

Word probabilities will be stored in a dictionary, as stated, with the word as the key name or index attached to a probability value. In this example, you are not using any 'smoothing', like add-one smoothing, which means there will be a lot of zeros and those words will not be included in the dictionary at all.

Use the 'trained' model to generate poems by Robert Frost (start with four lines at a time).

In [13]:
import numpy as np
import string

# Remove if you do not want reproducible results
np.random.seed(42)

In [14]:
initial = {} # First word of a sentence

first_order = {} # Second word only

second_order = {} # Word with at least two preceding words

In [3]:
# Standard function to remove punctuation in a line of text

def remove_punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))

In [4]:
# Create function that takes in a dictionary, key and value to populate said dictionary with key and value, i.e. collect words

def add2dict(d, k, v):
    if k not in d:
        d[k] = [] 
        d[k].append(v)

# [cat, cat, dog, dog, dog, dog, mouse, ...]

In [15]:
''' 
Using functions above, traverse the lines in text file to populate the empty dictionaries created above.
First tokenize the lines using the punctuation function, then loop through each word and add to: 
a) initial dictionary if it is the first word, and it doesn't exist in initial dictionary, add it and plus one to the word 
 count if it does.
b) second-order dictionary if it is the last word in a sentence (we don't want sentences to run on forever), add a fake 'END' 
 token as the value with the 'previous word, current word' as the key.
c) first-order dictionary if it is the second word given only one previous word, i.e. i == 1, where the previous word is the key
 and the current word is the value.
d) second-order dictionary for any word with two preceding words, where previous words are the key and the current word is the 
 value.
 
NOTE: The only dictionary with actual word counts will be the initial dictionary.
'''

for line in open('data/robert_frost.txt'):
    tokens = remove_punctuation(line.rstrip().lower()).split() 
    
    T = len(tokens)
    for i in range(T):
        t = tokens[i] 
        if i == 0:
            # Measure the counts of the first word
            initial[t] = initial.get(t, 0.) + 1
        else:
            t_1 = tokens[i - 1]
            
            if i == T - 1:
                # Add fake 'END' as value for word ending a line - (previous word, word) is the key
                add2dict(second_order, (t_1, t), 'END')                
            if i == 1:
                # Measure counts of second word (given only previous word t_1)
                add2dict(first_order, t_1, t)
            else:
                t_2 = tokens[i-2]
                add2dict(second_order, (t_2, t_1), t)

In [16]:
# Normalize the initial distribution (the only dictionary with word counts)

initial_total = sum(initial.values())

for t, c in initial.items():
    initial[t] = c / initial_total

In [17]:
dict(list(initial.items())[:3])

{'two': 0.005571030640668524,
 'and': 0.08983286908077995,
 'to': 0.034818941504178275}

In [18]:
dict(list(first_order.items())[:3])

{'two': ['roads'], 'and': ['sorry'], 'to': ['where']}

In [ ]:
{('two', 'roads'): ['diverged'],
 ('roads', 'diverged'): ['in'],
 ('diverged', 'in'): ['a']}

In [19]:
dict(list(second_order.items())[:3])

{('two', 'roads'): ['diverged'],
 ('roads', 'diverged'): ['in'],
 ('diverged', 'in'): ['a']}

In [7]:
# Function to convert input of list of words (tokens) into {'word1': 0.5, 'word2': 0.4, 'word3': 0.1}

def list2pdict(ts):
    d = {}
    n = len(ts) 
    
    # Find word count and add to value
    for t in ts:
        d[t] = d.get(t, 0.) + 1 
    
    # Find probability from word and count
    for t, c in d.items():
        d[t] = c / n 
        
    return d

In [20]:
# Replace word value in first_order dictionary to nested dictionary of word and probability

for t_1, ts in first_order.items(): 
    first_order[t_1] = list2pdict(ts)

In [24]:
dict(list(first_order.items())[:3])

{'two': {'roads': 1.0}, 'and': {'sorry': 1.0}, 'to': {'where': 1.0}}

In [21]:
# Replace word value in second_order dictionary to nested dictionary of word and probability

for k, ts in second_order.items():
    second_order[k] = list2pdict(ts)

In [23]:
dict(list(second_order.items())[:3])

{('two', 'roads'): {'diverged': 1.0},
 ('roads', 'diverged'): {'in': 1.0},
 ('diverged', 'in'): {'a': 1.0}}

**Now your dictionaries are ready to be randomly sampled from when generating lines of a poem.**

In [10]:
# Function to randomly sample words from a dictionary based on uniform distribution

def sample_word(d):
    # Draw random number between 0 and 1 (Print "d:", d) 
    p0 = np.random.random() 
    
    # Store the cumulative probabilities (Print "p0:", p0) 
    cumulative = 0 
    
    # Loop through tokens and probability values and increment cumulative sum
    for t, p in d.items():
        cumulative += p 
        if p0 < cumulative:
            return t 
        
    assert(False) # Should never get here!

In [11]:
def generate():
    for i in range(4): # Run loop 4 times (for 4 lines of the poem) 
        sentence = [] 
        
        # Sample one word from initial dictionary 
        w0 = sample_word(initial) 
        sentence.append(w0)
        
        # Sample one second word based on initial word (nested key under initial word)
        w1 = sample_word(first_order[w0]) 
        sentence.append(w1)
        
        # Sample second-order words until 'END' (infinite loop)
        while True:
            w2 = sample_word(second_order[(w0, w1)]) 
            if w2 == 'END':
                break         
            sentence.append(w2) 
            w0 = w1 
            w1 = w2 
        
        # Print tokens in single line by joining them
        print(' '.join(sentence))

In [27]:
generate()

though as for that the passing there
ill double theirs for both of them all aglitter
the darkest evening of the year
for hog reeve in march meeting here in warren


# Exercise 1:

### Determine the vocabulary size (V)

Use the method from text classifier session to create a vocabulary of words from the Robert Frost text data (word-to-index mapping dictionary).

### We know that pi has shape V, A1 has shape V x V, and A2 has shape V x V x V (full-dense arrays). In comparison, how many values are stored in our dictionaries?

This should demonstrate the advantage of using dictionaries.

# Exercise 2:

You can skip the step where you accumulate all the possible next words in a list, i.e. first and second-order dictionaries, 

e.g. [cat, cat, dog, dog, dog, ...]

Instead, like with initial state distribution, create the dictionary of counts directly as you loop through the data (you will no longer need `list2pdict()` function).